<a href="https://colab.research.google.com/github/fcolombo7/AN2DL-2020/blob/main/Final%20Notebooks/4_KerasTuner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **AN2DL** - Image classification challenge

* **Colombo** Filippo - 10559531
* **Del Vecchio** Giovanni - 10570682


## Keras Tuner



Configuration and constants:

In [ ]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd
from datetime import datetime

In [ ]:
#Random seed to make experiments reproducible
SEED = 1234
tf.random.set_seed(SEED)  

#Parameters
IMG_H, IMG_W = (400, 400)
BS = 32 #BATCH SIZE
VALIDATION_SPLIT = 0.2
DATA_AUGMENTATION = True

Load Google Drive to get the data and save the results:

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

cwd = os.getcwd()
drive_root_folder = '/content/drive/My Drive/ANN_project/'

Mounted at /content/drive


### Import the Mask Dataset

Check if the dataset has been already processed, otherwise unzip it.

**N.B.**: we are unzipping in the `/content` folder and not in `drive`.  
If you want to unzip in your drive folder, write `os.chdir(drive_root_folder)`



In [ ]:
#check if the dataset is already available
if not os.path.exists(cwd+'/MaskDataset'):
  !unzip '/content/drive/My Drive/ANN_project/artificial-neural-networks-and-deep-learning-2020.zip'
else:
  print('MaskDataset already loaded')

### Split the dataset

Definition of the `ImageDataGenerator` for _data augmentation_:

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

if DATA_AUGMENTATION:
    train_data_gen = ImageDataGenerator(rotation_range=20, 
                                        width_shift_range=0.3, 
                                        height_shift_range=0.3, 
                                        zoom_range=0.4, 
                                        horizontal_flip=True, #
                                        #brightness_range = [0.6, 1.5], 
                                        shear_range=10, 
                                        channel_shift_range=100, 
                                        fill_mode='reflect', 
                                        rescale=1./255)
else:
    train_data_gen = ImageDataGenerator(rescale=1./255)
    
valid_data_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
import json

dataset_dir = os.path.join(cwd, "MaskDataset")
training_dir = os.path.join(dataset_dir, "training")
with open(os.path.join(dataset_dir,"train_gt.json")) as f:
  dic = json.load(f)
  dataframe = pd.DataFrame(dic.items())
  dataframe.rename(columns = {0:'filename', 1:'class'}, inplace = True)
  dataframe = dataframe.sample(frac=1, random_state=SEED)
  
  tot_length = dataframe.shape[0]
  valid = dataframe.iloc[:int(np.ceil(tot_length * VALIDATION_SPLIT)),:] 
  train = dataframe.iloc[int(np.ceil(tot_length * VALIDATION_SPLIT)):,:]
  train["class"] = train["class"].astype('string')
  valid["class"] = valid["class"].astype('string')
  
  train_gen = train_data_gen.flow_from_dataframe(train,
                                               training_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)
  
  validation_gen = valid_data_gen.flow_from_dataframe(valid,
                                               training_dir,
                                               batch_size=BS,
                                               target_size=(IMG_H, IMG_W),
                                               class_mode='categorical',
                                               shuffle=True,
                                               seed=SEED)

Found 4491 validated image filenames belonging to 3 classes.
Found 1123 validated image filenames belonging to 3 classes.


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


### Define the search space and set up the tuner

#### Create the dataset objects

In [ ]:
num_classes = len(train_gen.class_indices)
num_classes
train_ds = tf.data.Dataset.from_generator(lambda: train_gen, 
                                          output_types=(tf.float32, tf.float32),
                                          output_shapes= ([None, IMG_H, IMG_W, 3], [None, num_classes]))

train_ds = train_ds.repeat()

valid_ds = tf.data.Dataset.from_generator(lambda: validation_gen,
                                          output_types = (tf.float32, tf.float32),
                                          output_shapes = ([None, IMG_H, IMG_W, 3], [None, num_classes]))
valid_ds = valid_ds.repeat()

#### Hypermodel definition

Use a builder function to define the model you set up for hypertuning.  
It contains the definition of the hyperparameter search space in addition to the model architecture.

In [ ]:
def tf_model_builder(hp):
  base_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', input_shape=(IMG_H,IMG_W,3))
  base_model.trainable = False
  model = tf.keras.Sequential()
  model.add(base_model)
  model.add(tf.keras.layers.Flatten())

  hp_units = hp.Int('units_first_layer', min_value = 256, max_value = 832, step = 64, default = 384)
  model.add(tf.keras.layers.Dense(units = hp_units, 
                                  activation = 'relu',
                                  kernel_regularizer = tf.keras.regularizers.l2(hp.Float('l2_first_layer', 0.005, 0.1, sampling='log'))
                                  ))
  
  model.add(tf.keras.layers.Dropout(hp.Float('dropout_first_layer', 0.0, 0.5, step=0.1, default=0.2)))


  for i in range(hp.Int('hidden_layers', 0, 2, default=1)):
    model.add(tf.keras.layers.Dense(units = hp.Int('units_hidden_layers', 64, 512, step=64, default=256),
                                    activation='relu',
                                    kernel_regularizer = tf.keras.regularizers.l2(hp.Float('l2_hidden_layers', 0.005, 0.1, sampling='log'))
                                  ))
    model.add(tf.keras.layers.Dropout(hp.Float('dropout_hidden_layers', 0.0, 0.5, step=0.1, default=0.2)))

  model.add(tf.keras.layers.Dense(units=3, activation='softmax'))

  model.compile( optimizer=tf.keras.optimizers.Adam(0.003),
      loss=tf.keras.losses.CategoricalCrossentropy(),
      metrics=['accuracy'])

  return model


#### Perform hyperparameter tuning

Download the KerasTuner package

In [ ]:
!pip install -q -U keras-tuner
import kerastuner as kt
import IPython


     |████████████████████████████████| 61kB 5.9MB/s 


Among the available tuner we choose **Hyperband**.  
*Hyperband tuning algorithm* trains a large number of models for a few epochs and carries forward only the top-performing half of models to the next round. 

In [ ]:
proj_name = 'VGG_TF_HyperTuning1'
tuner = kt.Hyperband(tf_model_builder,
                       objective = 'val_accuracy', 
                       max_epochs = 30,
                       factor = 3,
                       directory = os.getcwd(),
                       seed= SEED,
                       project_name = proj_name) 

58892288/58889256 [==============================] - 0s 0us/step


Define a custom callback that keeps clean the cell output at each step.

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

Start the search.

In [ ]:
tuner.search(train_ds,
               epochs = 30,
               validation_data = valid_ds,
               steps_per_epoch = len(train_gen),
               validation_steps = len(validation_gen),
               callbacks = [ClearTrainingOutput()])

  # Get the optimal hyperparameters
  best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]
  out = f"""****Hyperband serach completed****\n
  **********************************\n
  Optimal parameters found:\n
  units_first_layer: {best_hps.get('units_first_layer')}\n
  l2_first_layer: {best_hps.get('l2_first_layer')}\n
  dropout_first_layer: {best_hps.get('dropout_first_layer')}\n
  hidden_layers: {best_hps.get('hidden_layers')}\n
  units_hidden_layers: {best_hps.get('units_hidden_layers')}\n
  l2_hidden_layers: {best_hps.get('l2_hidden_layers')}\n
  dropout_hidden_layers: {best_hps.get('dropout_hidden_layers')}"""
  
  print(out)

  with open(os.path.join(drive_root_folder, project_name+'.txt'), 'w') as f:
            f.write(out)


Epoch 1/2
71/71 [==============================] - 120s 2s/step - loss: 11.4246 - accuracy: 0.4130 - val_loss: 3.6284 - val_accuracy: 0.5690
Epoch 2/2
 7/71 [=>............................] - ETA: 1:24 - loss: 3.6194 - accuracy: 0.4420